# What needs to be done (general plan):

# On: http://www.digital.nyc/startups
- http://www.digital.nyc/startups
- http://www.digital.nyc/startups?page=1
- http://www.digital.nyc/startups?page=380

# Out of each page, extract out:
- Startup name : Next Island LLC
- Website on the digital thing: http://www.digital.nyc/startups/next-island-llc

# Go to the website: e.g. 
- http://www.digital.nyc/startups/next-island-llc

# Pull the "Script" via beautiful soup:
- <script>jQuery.extend(Drupal.settings, ...

# Select out the lat & long values
"center":{"lat":"","lon":"","geocode":"Find my location"}},"data":{"type":"Point","coordinates":[-74.006,40.7144],"

# Doing the work

## Import Directories

In [36]:
import requests
import json
from bs4 import BeautifulSoup
import pprint

In [80]:
url = "http://www.boxofficemojo.com/intl/china/yearly/?yr=2015&p=.htm"

In [81]:
def extract_location_data(url):
    response = requests.get(url)
    page_html = response.text
    soup = BeautifulSoup(page_html)
    
    # Retrieve all of the script tags
    blah = soup('tbody')
    print blah

In [ ]:
extract_location_data(url)

## Generate the list of startup pages

In [12]:
digital_nyc_startups_pages_list = ["http://www.digital.nyc/startups"]

for i in range(1,380):
    website = "http://www.digital.nyc/startups?page=" + str(i)
    digital_nyc_startups_pages_list.append(website)

In [13]:
print digital_nyc_startups_pages_list[:3]

['http://www.digital.nyc/startups', 'http://www.digital.nyc/startups?page=1', 'http://www.digital.nyc/startups?page=2']


## Go through all those http://www.digital.nyc/startups/ links and compile a list of individual companies

In [14]:
def get_company_links(startup_url):
    '''This takes some webpage like http://www.digital.nyc/startups?page=1
    and returns a list of tuples (company_data_listof the form (company name, company url).
    That object is called companies_list'''
    response = requests.get(startup_url)
    page_html = response.text
    soup = BeautifulSoup(page_html)
    # Retrieve all of the link tags
    links_collection = soup.find_all('a')
    companies_list = []
    for link in links_collection:
        try:
            # There are some links without href -- this keeps the code from 
            # breaking when it gets to them
            #print link["href"]
            if "/startups/" in link["href"]:    #This selects only htose links of startups
                #print True
                company_url = "http://www.digital.nyc/" + str(link["href"])
                #print company_url
                company_name = link["href"][10:]
                #print company_name
                company_tuple = (company_name,company_url)
                if company_tuple not in companies_list:
                    companies_list.append(company_tuple)
        except:
            pass
    return companies_list

In [15]:
big_list_of_company_websites = []

# Warning the next step is going to take a really long time (it actually scrapes the web)

In [16]:
for webpage in digital_nyc_startups_pages_list:
    #print webpage
    #print get_company_links(fwebpage)
    big_list_of_company_websites = big_list_of_company_websites + get_company_links(webpage)
    #print

In [17]:
print len(big_list_of_company_websites)

7494


In [18]:
big_list_of_company_websites[0:10]

[('genesis', 'http://www.digital.nyc//startups/genesis'),
 ('yank-technologies-inc',
  'http://www.digital.nyc//startups/yank-technologies-inc'),
 ('cinematcher', 'http://www.digital.nyc//startups/cinematcher'),
 ('1-atelier-llc', 'http://www.digital.nyc//startups/1-atelier-llc'),
 ('first-wall-street', 'http://www.digital.nyc//startups/first-wall-street'),
 ('httpwwwbluemailmediacom',
  'http://www.digital.nyc//startups/httpwwwbluemailmediacom'),
 ('studytree', 'http://www.digital.nyc//startups/studytree'),
 ('heed-media', 'http://www.digital.nyc//startups/heed-media'),
 ('pips-island', 'http://www.digital.nyc//startups/pips-island'),
 ('raleigh-and-drake', 'http://www.digital.nyc//startups/raleigh-and-drake')]

In [19]:
# Saving it to csv just in case...
import csv

with open('big_list_of_company_websites.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(big_list_of_company_websites)

In [20]:
big_list_of_company_websites1 = big_list_of_company_websites[0:1000]
big_list_of_company_websites2 = big_list_of_company_websites[1000:2000]
big_list_of_company_websites3 = big_list_of_company_websites[2000:3000]
big_list_of_company_websites4 = big_list_of_company_websites[3000:4000]
big_list_of_company_websites5 = big_list_of_company_websites[4000:5000]
big_list_of_company_websites6 = big_list_of_company_websites[5000:6000]
big_list_of_company_websites7 = big_list_of_company_websites[6000:7000]
big_list_of_company_websites8 = big_list_of_company_websites[7000:]

1000

# Goes to Website, pulls out script, extracts out coordinates

In [109]:
def extract_location_data(url):
    response = requests.get(url)
    page_html = response.text
    soup = BeautifulSoup(page_html)
    
    # Retrieve all of the script tags
    scripts_collection = soup('script')
    # Finds script associated with the google maps thing in the top right
    #for i in range(0,len(scripts_collection)):
    #    print i
    #    print scripts_collection[i]
    google_script_BSTag = scripts_collection[8]
    # This is a bunch of stuff to make the script easy to work with
    #print google_script_BSTag
    google_script_string = str(google_script_BSTag)
    google_script_string_dict_portion = google_script_string[39:len(google_script_string)-11]    # Removes the javascript/jquery grammer to just get the JSON portion

    google_script_dict = json.loads(google_script_string_dict_portion)
    #print google_script_dict['geofieldMap']

    #pp = pprint.PrettyPrinter(depth=6)
    #pp.pprint(google_script_dict)
    # (There was a bunch of steps here nec. to determine where the lat long info was contained)

    
    # http://stackoverflow.com/questions/17106819/accessing-python-dict-values-with-the-key-start-characters
    # It seems the key that looks like the following changes it's numbers ever time there's a new point. Since
    # I can't predict the numbers, I'll just have create the key on the fly.
    # ["geofield-map-entity-node-965-field-geo-location"]

    geofield_etc_key = google_script_dict['geofieldMap'].keys()[0]
    lat_long_list = google_script_dict['geofieldMap'][geofield_etc_key]["data"]["coordinates"]
    return lat_long_list


In [110]:
extract_location_data("http://www.digital.nyc//startups/better-schools-everyone")

[-74.006, 40.7144]

In [93]:
big_list_of_company_websites4[0:10]

[('better-schools-everyone',
  'http://www.digital.nyc//startups/better-schools-everyone'),
 ('foxotic', 'http://www.digital.nyc//startups/foxotic'),
 ('cheater-beater', 'http://www.digital.nyc//startups/cheater-beater'),
 ('vowched', 'http://www.digital.nyc//startups/vowched'),
 ('domainpaid-inc', 'http://www.digital.nyc//startups/domainpaid-inc'),
 ('prax-algorithm-designs',
  'http://www.digital.nyc//startups/prax-algorithm-designs'),
 ('playdin-llc', 'http://www.digital.nyc//startups/playdin-llc'),
 ('ovenhotfoodgroup', 'http://www.digital.nyc//startups/ovenhotfoodgroup'),
 ('educo-kids-inc', 'http://www.digital.nyc//startups/educo-kids-inc'),
 ('palace-design', 'http://www.digital.nyc//startups/palace-design')]

In [68]:
''' Commenting this out just to make sure peopel don't accidently erase the data
full_list_company_data = []
full_list_company_data1 = []
full_list_company_data2 = []
full_list_company_data3 = []
full_list_company_data4 = []
full_list_company_data5 = []
full_list_company_data6 = []
full_list_company_data7 = []
full_list_company_data8 = []
'''
# Estimated it may take 30-1hr
# #1 - Started at 11:43 -- done by 12:16
# #2 - Started at 12:45 -- done by 1:07
# #3 - Started at 1:08 -- done by 
# #4 - Started at  -- done by 
# #5 - Started at  -- done by 
# #6 - Started at  -- done by 
# #7 - Started at  -- done by 
# #8 - Started at  -- done by 

In [ ]:
''' This worked to get the first 3 sets of distance data but seemed to stop afterwards'''

counter = 0
for company_name_and_link in big_list_of_company_websites5:
    name = company_name_and_link[0]
    link = company_name_and_link[1]
    counter += 1
    print counter,
    try:
        location = extract_location_data(link)
        longitude = location[0]
        latitude = location[1]
        company_tuple_full = (name,link,latitude,longitude)
        full_list_company_data5.append(company_tuple_full)
    except:
        pass



1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58

In [112]:
print "Total Websites to Scrape:", len(big_list_of_company_websites1), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data1)
print "Total Websites to Scrape:", len(big_list_of_company_websites2), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data2)
print "Total Websites to Scrape:", len(big_list_of_company_websites3), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data3)
print "Total Websites to Scrape:", len(big_list_of_company_websites4), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data4)
print "Total Websites to Scrape:", len(big_list_of_company_websites5), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data5)
print "Total Websites to Scrape:", len(big_list_of_company_websites6), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data6)
print "Total Websites to Scrape:", len(big_list_of_company_websites7), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data7)
print "Total Websites to Scrape:", len(big_list_of_company_websites8), "Total Companies w/ scraped Lat/Long Data", len(full_list_company_data8)


Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 939
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 980
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 913
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 999
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 0
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 0
Total Websites to Scrape: 1000 Total Companies w/ scraped Lat/Long Data 0
Total Websites to Scrape: 494 Total Companies w/ scraped Lat/Long Data 0
